# Mount and Imports

In [ ]:
#drive.flush_and_unmount()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
pip install wandb -qq

In [ ]:
pip install imbalanced-learn

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.tensorboard as tb
from torch.nn import functional as F


import os
import numpy as np
import math
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import wandb

In [ ]:
# Set the device to use for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Autoencoder

In [ ]:
'''   Dataset   '''

'''     imports     '''
import os
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import tqdm
import torch

'''   custom dataset with trajectories    '''
class CustomDataset(Dataset):
    def __init__(self, dataset_path):

        self.data = []
        self.trajectories = []
        self.labels = []

        self.labels_codec = LabelEncoder()

        self._init_dataset(dataset_path)



    def __len__(self):
        return len(self.trajectories)


    def __getitem__(self, idx):
        class_id, trajectory = self.data[idx]

        trajectory = torch.Tensor(trajectory)

        # return both trajectory and class id
        return trajectory, class_id

    def _init_dataset(self, dataset_path):

        list_behaviors = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']
        #list_behaviors = [item for item in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, item))]
        self.labels_codec.fit(list_behaviors)

        for root, dirs, filenames in os.walk(dataset_path, topdown=False):
            for filename in filenames:
                behavior = filename[:filename.find("_")]
                behavior_int = self.label_to_int(behavior)
                self.labels.append(behavior_int)

                filename_path = root + "/" + filename
                df = pd.read_csv(filename_path)

                trajectory = []
                for i in range(len(df)):
                    traj_point = df.loc[i].tolist()
                    trajectory.append(traj_point)

                self.trajectories.append(trajectory)

                data = [behavior_int, trajectory]
                self.data.append(data)

    def label_to_int(self, label):
        value = self.labels_codec.transform([label])
        return value[0]

    def int_to_label(self, value):
        label = self.labels_codec.inverse_transform([value])
        return label[0]

In [ ]:
'''   AutoEncoder Model   '''

# imports
from typing import List
import torch.nn as nn
import torch


# residual block for encoder
class ResConvBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True
    ) -> None:
        super(ResConvBlock, self).__init__()

        # 1d pooling with kernel = 2, it returns indices
        self.pool = nn.MaxPool1d(2, return_indices=True)
        self.relu = nn.ReLU(inplace=False)

        # encoder block
        self.block = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, bias=bias, padding = 1),
            nn.ReLU(inplace=False),
        )

        # convolutional layer shared by the residual block and the skip connection
        self.conv1 = self.block[0]

    def forward(self, x):
        identity = self.conv1(x)
        x = self.block(x)
        x = x + identity
        x = self.relu(x)
        return self.pool(x)


# residual block for decoder
class ResTConvBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True
    ) -> None:
        super(ResTConvBlock, self).__init__()
        self.relu = nn.ReLU(inplace=False)

        # batch normalization
        self.bn = nn.BatchNorm1d(out_channels)

        # decoder block
        self.block = nn.Sequential(
            nn.ConvTranspose1d(in_channels, out_channels, kernel_size=3, stride=1, bias=bias, padding=1),
            nn.ReLU(inplace=False),
        )

        # transpose layer shared by the skip and the decoder block
        self.devonc3 = self.block[0]

    def forward(self, x):
        identity = self.devonc3(x)
        x = self.block(x)
        x = x + identity
        x = self.relu(x)
        return self.bn(x)


class AutoEncoder(nn.Module):
    def __init__(
        self,
        layers: List[int] =  [64, 128],
        latent_space: int = 2048,
        bias: bool = True
    ) -> None:
        super(AutoEncoder, self).__init__()
        self.layers = layers

        # definition encoder blocks
        self.enc_block1 = ResConvBlock(3, layers[0])
        self.enc_block2 = ResConvBlock(layers[0], layers[1])

        # definition linear layer for the latent space
        self.encoder_fc = nn.Linear(layers[1] * 3, latent_space, bias=bias)

        # definition decoder blocks
        self.dec_block2 = ResTConvBlock(layers[1], layers[0])
        self.dec_block1 = ResTConvBlock(layers[0], 3)

        # definition linear layer for the latent space
        self.decoder_fc = nn.Linear(latent_space, layers[1] * 3, bias=bias)

        # definition batch norm for normalizing the latent space
        self.dec_bn = nn.BatchNorm1d(layers[1])

        self.initialize_weights()


    # initialize the weights
    def initialize_weights(self):
        for m in self.modules():
          if isinstance(m, nn.Conv1d) or isinstance(m, nn.ConvTranspose1d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')

          elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    # encoder function
    def encoder(self, x: torch.Tensor) -> torch.Tensor:
        x, self.i1 = self.enc_block1(x)
        x, self.i2 = self.enc_block2(x)
        x = self.encoder_fc(torch.flatten(x, 1))
        return x

    def freeze_encoder(self):
        self.enc_block1.requires_grad = False
        self.enc_block2.requires_grad = False
        self.encoder_fc.requires_grad = False
        print("Encoder is frozen")

    # decoder function
    def decoder(self, x: torch.Tensor) -> torch.Tensor:
        x = self.decoder_fc(x)
        x = self.dec_bn(x.view([-1, self.layers[1], 3]))

        self.unpool2 = torch.nn.MaxUnpool1d(2)
        x = self.unpool2(x, self.i2)
        x = self.dec_block2(x)

        self.unpool1 = torch.nn.MaxUnpool1d(2)
        x = self.unpool1(x, self.i1, output_size = [13])
        x = self.dec_block1(x)
        return x


    # forward function
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        latent_code = self.encoder(x)
        rec = self.decoder(latent_code)
        return rec, latent_code

In [ ]:
'''   utils   '''

class Metric(object):
    def __init__(self, name):
        self.name = name
        self.sum = torch.tensor(0.)
        self.n = torch.tensor(0.)

    def update(self, val):
        self.sum += val.detach().cpu()
        self.n += 1

    @property
    def avg(self):
        return self.sum / self.n

def random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def log(s, nl=True):
    print(s, end='\n' if nl else '')

In [ ]:
from tqdm import tqdm
from torch.nn.functional import mse_loss, l1_loss

'''   training    '''
def train_auto(autoencoder, dataloader, epoch, reconstruction_loss, optimizer, scheduler, batch_size):

    autoencoder.train()

    # initialize metrics
    trainMSE = Metric('trainMSE')

    # start batch training
    progress_bar = tqdm(dataloader, desc='description')

    for data in progress_bar:

            traj = data[0]
            labels = data[1]
            traj = traj.to(device)

            x = torch.transpose(traj, 2,1)

            reconstructed_traj, latent_codes = autoencoder(x)
            mse_loss = reconstruction_loss(reconstructed_traj, x)
            batch_loss = mse_loss

            # update metrics
            trainMSE.update(mse_loss)


            optimizer.zero_grad()
            batch_loss.div_(math.ceil(float(len(traj)) / batch_size))
            batch_loss.backward()
            optimizer.step()


    train_log = {'Train MSE': trainMSE.avg.item()}


    return train_log

## Training Autoencoder

In [ ]:
'''   imports   '''
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
import wandb
from sklearn.neighbors import KNeighborsClassifier
import numpy
import warnings
from sklearn.manifold import TSNE
from numpy import reshape
import seaborn as sns

'''   main    '''
def start(lr=10e-4, batch_size=64, n_worders=2, epochs=10, layers=[64,128],latent_space=2048, bias=True, dataset_path="", wandb_update=False, seed=3):

    # ignore warnings
    warnings.filterwarnings("ignore")

    print("initialization")


    # initialize model with dataparallel (according to the # of GPUs available)
    autoencoder = torch.nn.DataParallel(AutoEncoder(
        layers = [64, 128],
        latent_space = 512,
        bias = True
        ).cuda(),
        device_ids=range(torch.cuda.device_count()))


    # initialize random seeds
    random_seed(seed)

    # reconstruction loss, optimizer and scheduler
    reconstruction_loss = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr = lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                           mode='min',
                                                           factor=0.1,
                                                           patience=8,
                                                           cooldown=0,
                                                           verbose=True)
    device = "cuda:0"

    if wandb_update:
      # save model inputs and hyperparameters
      config = {
        "lr": lr,
        "batch_size": batch_size,
        "epochs": epochs,
        "layers": layers,
        "latent_space": latent_space,
      }

      # start a new wandb run
      run = wandb.init(project='project_name', entity='entity_name', config=config)

      # log gradients and model parameters
      wandb.watch(autoencoder)

    print("dataset")

    # load dataset
    dataset = CustomDataset(dataset_path)
    print(len(dataset))

    # dataloader
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    # training
    print('start training')

    for epoch in range(epochs):
        print('-----------------')

        print('train')
        # train
        train_log = train_auto(autoencoder, loader, epoch, reconstruction_loss, optimizer, scheduler, batch_size)


        log('--------------------')
        log('Epoch: {}'.format(epoch))
        log('Train MSE:{}'.format(train_log['Train MSE']))

        '''
        if train_log['Train MSE'] < 0.00058:
          path = "/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/Checkpoint/"
          name = "autoencoderlin" + str(100. * train_log['Train MSE'])
          torch.save({
              'epoch': epoch+1,
              'model_state_dict': autoencoder.state_dict(),
              'optimizer_state_dict': optimizer.state_dict()
          }, f'{path}/{name}.pth')

        '''
        if wandb_update:
          wandb.log({'Epochs': epoch + 1,
                    'Train Loss': train_log['Train MSE']})


        scheduler.step(train_log['Train MSE'])
    if wandb_update:
      run.finish()

In [ ]:
dataset_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/SLB'

lr = 10e-4
batch_size = 64
n_workers = 2
epochs = 100
layers = [64, 128],
latent_space = 512,
bias = True
wandb_update = True
seed = 3

start(lr=lr, batch_size=batch_size, n_worders=n_workers, epochs=epochs, layers=layers,latent_space=latent_space, bias=bias, dataset_path=dataset_path, wandb_update=wandb_update, seed=seed)

## Eval Autoencoder


In [ ]:
'''   eval    '''
def eval(autoencoder, dataloader, reconstruction_loss, dataset):

    autoencoder.eval()

    testMSE = Metric('testMSE')

    rec_traj_d = {}

    # start batch testing
    progress_bar = tqdm(dataloader, desc='description')
    for data in progress_bar:

      traj = data[0]
      labels = data[1]
      traj = traj.to(device)

      x = torch.transpose(traj, 2,1)

      #with torch.no_grad():
      reconstructed_traj, latent_codes = autoencoder(x)
      mse_loss = reconstruction_loss(reconstructed_traj, x)


      # update metrics
      testMSE.update(mse_loss)

    test_log = {'Test MSE': testMSE.avg.item()}

    return test_log
    #return test_log, rec_traj_d



In [ ]:
'''   imports   '''
import pandas as pd
from tqdm import tqdm

# autoencoder checkpoint path
path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/Checkpoint/checkpoint.pth'

# initialize autoencoder
autoencoder = torch.nn.DataParallel(AutoEncoder(
            layers = [64, 128],
            latent_space = 512,
            bias = True
            ).cuda(),
            device_ids=range(torch.cuda.device_count()))

# load model state to autoencoder
checkpoint = torch.load(path)
autoencoder.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']

# load dataset
dataset_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/SLB/'
dataset = CustomDataset(dataset_path)

loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
print(len(dataset))

# reconstruction loss
reconstruction_loss = nn.MSELoss()

# eval step
test_log = eval(autoencoder, loader, reconstruction_loss, dataset)
#test_log, rec_traj_d = eval(autoencoder, loader, reconstruction_loss, dataset)

# print mse error
log('Test Loss: {}'.format(test_log['Test MSE']))

'''
# save some reconstructed trajectories
for key in rec_traj_d:

  # save trajectory to csv file
  rec_traj = rec_traj_d.get(key)
  rec_traj = rec_traj.cpu()
  rec_traj = rec_traj.detach().numpy()

  x = rec_traj[0][0]
  y = rec_traj[0][1]
  z = rec_traj[0][2]

  df = pd.DataFrame(list(zip(x, y, z)), columns=['x','y','ratio_wh'])

  file_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/' + key + '.csv'
  df.to_csv(file_path, index=False)
'''

# Latent Codes


In [ ]:
'''   eval    '''
def eval(autoencoder, dataloader, reconstruction_loss, dataset):

    autoencoder.eval()

    testMSE = Metric('testMSE')

    label_latent_list = []

    # start batch testing
    progress_bar = tqdm(dataloader, desc='description')
    for data in progress_bar:

      label_latent = []

      traj = data[0]
      labels = data[1]
      traj = traj.to(device)


      x = torch.transpose(traj, 2,1)

      with torch.no_grad():
          reconstructed_traj, latent_codes = autoencoder(x)
          mse_loss = reconstruction_loss(reconstructed_traj, x)



      label_latent = [labels, latent_codes]

      # update metrics
      testMSE.update(mse_loss)

    test_log = {'Test MSE': testMSE.avg.item()}

    return test_log, label_latent

In [ ]:
'''   imports   '''
import pandas as pd

# autoencoder checkpoint path
path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/Checkpoint/checkpoint.pth'

# initialize autoencoder
autoencoder = torch.nn.DataParallel(AutoEncoder(
            layers = [64, 128],
            latent_space = 512,
            bias = True
            ).cuda(),
            device_ids=range(torch.cuda.device_count()))

# load model state to autoencoder
checkpoint = torch.load(path)
autoencoder.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']

# load dataset
dataset_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/Divided/train'
dataset = CustomDataset(dataset_path)
print(len(dataset))

loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False, num_workers=2, pin_memory=True)


# reconstruction loss
reconstruction_loss = nn.MSELoss()

# eval step
test_log, label_latent = eval(autoencoder, loader, reconstruction_loss, dataset)

# print mse error
log('Test Loss: {}'.format(test_log['Test MSE']))

# save csv with latent codes
labels = label_latent[0]
latents = label_latent[1]

label_names = []
for label in labels:
  label_name = str(dataset.int_to_label(int(label)))
  label_names.append(label_name)

latent_codes = []
for latent in latents:
  latent = latent.cpu()
  latent = latent.detach().numpy()
  print("latent", type(latent))
  a = str(latent).replace("[", "")
  a = a.replace("]","")
  a = a.replace("  ", " ")
  a = a.strip()
  latent_codes.append(a)


df = pd.DataFrame(list(zip(label_names,latent_codes)), columns=['label','latent_code'])

print(df)

file_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/' + 'train' + '.csv'
df.to_csv(file_path, index=False)

# MLP

In [ ]:
'''     imports     '''
import imblearn
from array import array
import numpy as np
from collections import Counter
from imblearn.over_sampling import SMOTE

In [ ]:
'''   Dataset   '''

'''     imports     '''
import os
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import torch

'''   custom dataset with latent codes    '''
class CustomDatasetLatent(Dataset):
    def __init__(self, dataset_path):

        self.data = []
        self.latent_codes = []
        self.labels = []

        self.labels_codec = LabelEncoder()

        self._init_dataset(dataset_path)

    def __len__(self):
        return len(self.latent_codes)


    def __getitem__(self, idx):
        class_id, latent_code = self.data[idx]

        latent_code = torch.Tensor(latent_code)

        # return both trajectory and class id
        return latent_code, class_id

    def _init_dataset(self, dataset_path):

        list_behaviors = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']
        self.labels_codec.fit(list_behaviors)

        df = pd.read_csv(dataset_path)
        self.labels = [self.label_to_int(i) for i in df['label'].tolist()]

        self.latent_codes = df['latent_code'].tolist()

        for label, latent in zip(self.labels, self.latent_codes):

          latent = list(latent.split(" "))
          code = []

          for l in latent:
            if l != '':
                if '\n' in l:
                  l = l.replace('\n','')

                code.append(float(l))

          data = [label, code]

          self.data.append(data)


    def label_to_int(self, label):
        value = self.labels_codec.transform([label])
        return value[0]

    def int_to_label(self, value):
        label = self.labels_codec.inverse_transform([value])
        return label[0]

In [ ]:
'''   MLP model   '''

import torch.nn as nn

class MLP(nn.Module):
    def __init__(
            self,
            encoder_latent_space,
            output_classes
    ):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(encoder_latent_space, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, output_classes),
        )

    def forward(self, x):
        return self.mlp(x)

In [ ]:
'''   Focal Loss with alpha equal to inverse of class frequencies  '''
'''https://saturncloud.io/blog/how-to-use-class-weights-with-focal-loss-in-pytorch-for-imbalanced-multiclass-classification/#:~:text=Focal%20loss%20works%20by%20down,performance%20on%20the%20minority%20class.'''
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        min = 1e-8
        max = 1 - min
        inputs = torch.clamp(inputs, min, max)
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        pt = torch.exp(-ce_loss)
        targets = targets.cpu()
        pt = pt.cpu()
        ce_loss = ce_loss.cpu()
        loss = (self.alpha[targets] * (1 - pt) ** self.gamma * ce_loss).mean()
        return loss

In [ ]:
'''   Focal Loss with fixed alpha  '''
'''https://discuss.pytorch.org/t/focal-loss-for-imbalanced-multi-class-classification-in-pytorch/61289/9'''
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, outputs, targets):
        min = 1e-8
        max = 1 - min
        outputs = torch.clamp(outputs, min, max)

        ce_loss = torch.nn.functional.cross_entropy(outputs, targets, reduction='none') # important to add reduction='none' to keep per-batch-item loss
        ce_loss = ce_loss.cpu()
        pt = torch.exp(-ce_loss)
        targets = targets.cpu()
        focal_loss = (self.alpha * (1-pt)**self.gamma * ce_loss).mean() # mean over the batch

        return focal_loss

In [ ]:
'''   utils   '''

class Metric(object):
    def __init__(self, name):
        self.name = name
        self.sum = torch.tensor(0.)
        self.n = torch.tensor(0.)

    def update(self, val):
        self.sum += val.detach().cpu()
        self.n += 1

    @property
    def avg(self):
        return self.sum / self.n

def random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def log(s, nl=True):
    print(s, end='\n' if nl else '')

In [ ]:
'''   Dataset   '''

'''     imports     '''
import os
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import torch

'''   custom dataset with latent codes    '''
class CustomDatasetSmote(Dataset):
    def __init__(self, x, y):

        self.data = []
        self.latent_codes = []
        self.labels = []

        self.labels_codec = LabelEncoder()

        self._init_dataset(x, y)

    def __len__(self):
        return len(self.latent_codes)


    def __getitem__(self, idx):
        class_id, latent_code = self.data[idx]

        latent_code = torch.Tensor(latent_code)

        # return both trajectory and class id
        return latent_code, class_id

    def _init_dataset(self, x, y):

        list_behaviors = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']
        self.labels_codec.fit(list_behaviors)

        for label, latent in zip(y,x):
            data = [label, latent]
            self.data.append(data)

        self.labels = y
        self.latent_codes = x


    def label_to_int(self, label):
        value = self.labels_codec.transform([label])
        return value[0]

    def int_to_label(self, value):
        label = self.labels_codec.inverse_transform([value])
        return label[0]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score


'''   training    '''
def train(model, optimizer, scheduler, criterion, epochs, device, train_loader, test_loader, save_weights_name, batch_size, save_model=False, wandb_update=False):
    model.to(device)


    for e in range(epochs):

        '''       training    '''
        print('training')
        model.train()

        trainF = Metric('trainF')

        progress_bar = tqdm(train_loader, desc='description')

        overall_preds = []
        overall_targets = []
        softmax = torch.nn.Softmax(dim=1)


        for code, targets in progress_bar:

            code = code.to(device)
            targets = targets.to(device)

            # forward
            outputs = model(code)

            # zero grad
            optimizer.zero_grad()

            # compute loss
            loss = criterion(outputs, targets)

            loss.div_(math.ceil(float(len(code)) / batch_size))

            # softmax tha outputs and take the max index (argmax), that is the final predicted class
            preds = torch.argmax(softmax(outputs), dim=1).cpu()

            # save results and targets for accuracy metrics
            overall_preds += preds.tolist()
            overall_targets += targets.cpu().tolist()

            # backpropagate the error
            loss.backward()

            # clip gradient
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # update weights
            optimizer.step()

            # update loss
            trainF.update(loss)


        model.eval()
        train_accuracy = accuracy_score(overall_targets, overall_preds)
        train_f1 = f1_score(overall_targets, overall_preds, average='macro')




        # save loss for epoch
        train_log = {'Train F': trainF.avg.item(),
                     'Train A': train_accuracy,
                     'Train F1': train_f1}

        log('--------------------')
        log('Epoch: {}'.format(e))
        log('Train Focal Loss:{}'.format(train_log['Train F']))
        log('Train Accuracy:{}'.format(train_accuracy))
        log('Train F1 score:{}'.format(train_f1))


        '''       test      '''
        print("test")
        model.eval()
        testF = Metric('testF')

        with torch.no_grad():
          progress_bar = tqdm(test_loader, desc='description')
          overall_preds = []
          overall_targets = []
          softmax = torch.nn.Softmax(dim=1)

          for code, targets in progress_bar:
              code = code.to(device)
              targets = targets.to(device)

              # forward
              outputs = model(code)

              # compute testing loss
              loss = criterion(outputs, targets)

              # softmax tha outputs and take the max index (argmax), that is the final predicted class
              preds = torch.argmax(softmax(outputs), dim=1).cpu()

              # save results and targets for accuracy metrics
              overall_preds += preds.tolist()
              overall_targets += targets.cpu().tolist()

              testF.update(loss)

          test_accuracy = accuracy_score(overall_targets, overall_preds)
          test_f1 = f1_score(overall_targets, overall_preds, average='macro')

          geo_w = geometric_mean_score(overall_targets, overall_preds, average='weighted')
          geo_m = geometric_mean_score(overall_targets, overall_preds, average='macro')

          test_log = {'Test F': testF.avg.item(),
                       'Test A': test_accuracy,
                      'Test F1': test_f1,
                      'Test Geo W': geo_w,
                      'Test Geo M': geo_m}

          log('Test Focal Loss:{}'.format(test_log['Test F']))
          log('Test Accuracy:{}'.format(test_log['Test A']))
          log('Test F1:{}'.format(test_log['Test F1']))
          log('Test Geo Weighted:{}'.format(test_log['Test Geo W']))
          log('Test Geo Macro:{}'.format(test_log['Test Geo M']))


        # save model
        #if save_model:


        if test_f1 >= 0.4:
            path = "/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/Checkpoint/"
            #name = "MLP" + str(100. * train_log['Train A'])
            name = "MLPMS"  + "_" + str(100. * test_f1)
            torch.save({
                'epoch': e+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            },f'{path}/{name}.pth')



        # update wandb
        if wandb_update:
          wandb.log({'Epochs': e + 1,
                    'Train CE Loss': train_log['Train F'],
                     'Train Accuracy': train_log['Train A'],
                     'Train F1': train_log['Train F1'],
                     'Test CE Loss': test_log['Test F'],
                     'Test Accuracy': test_log['Test A'],
                     'Test F1': test_log['Test F1'],
                     'Test Geo Weighted': test_log['Test Geo W'],
                     'Test Geo Macro':test_log['Test Geo M']})

        scheduler.step(train_log['Train F'])

In [ ]:
'''   main    '''
def main():
    lr = 5e-5
    batch_size = 32
    n_workers = 2
    epochs = 250
    wandb_update = True
    weight_decay = 1e-3
    gamma = 2

    latent_space_dim = 512

    seed = 3

    print("begin")
    # initialize random seeds
    random_seed(seed)

    mlp = MLP(
        encoder_latent_space = latent_space_dim,
        output_classes = 12
    )

    #df_weight = pd.read_csv('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/train_class_weight.csv')
    #class_weights = df_weight['weight'].tolist()

    #class_weights = torch.FloatTensor(class_weights)


    #criterion = FocalLoss(alpha=class_weights, gamma=gamma)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mlp.parameters(), lr = lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                           mode='min',
                                                           factor=0.1,
                                                           patience=10,
                                                           cooldown=0,
                                                           verbose=True)

    device = "cuda:0"

    if wandb_update:
      # save model inputs and hyperparameters
      config = {
        "lr": lr,
        "batch_size": batch_size,
        "epochs": epochs,
        "latent_space": latent_space_dim,
        "loss": 'CE Loss',
        "weight_decay": weight_decay,
        "gamma": gamma
      }

      # start a new wandb run
      run = wandb.init(project='project_name', entity='entity_name', config=config)

      # log gradients and model parameters
      wandb.watch(mlp)

    print("dataset")
    # load datasets
    # train dataset
    path_train = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/train.csv'
    dataset_train = CustomDatasetLatent(path_train)
    #dataloader_train = DataLoader(dataset_train, shuffle = True, batch_size=batch_size)
    #print(len(dataset_train))

    # test dataset
    path_test = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/test.csv'
    dataset_test = CustomDatasetLatent(path_test)
    dataloader_test = DataLoader(dataset_test, shuffle = True, batch_size=batch_size, num_workers=n_workers, pin_memory=True)
    print(len(dataset_test))

    # smote
    to_sample = {1:33, 3: 114, 4: 18, 7: 9, 11: 57}

    oversample = SMOTE(k_neighbors=2, sampling_strategy=to_sample)

    x = []
    y = []
    for i in range(len(dataset_train)):
      array = dataset_train[i][0].numpy()
      x.append(array)
      y.append(dataset_train[i][1])

    from collections import Counter
    y_counter = Counter(y)
    print(y_counter)

    total_samples = len(y)

    # for each class, calculate weight
    class_weights = []
    for beh, count in y_counter.items():
      weight = 1/ (count /total_samples)
      class_weights.append(weight)

    print(class_weights)


    res = np.array(x)
    res = res.reshape(13489, -1)
    res, y = oversample.fit_resample(res,y)

    #list_y  = y.tolist()
    from collections import Counter
    print(Counter(y))

    dataset = CustomDatasetSmote(res, y)
    dataloader_train = DataLoader(dataset, shuffle = True, batch_size=batch_size)
    print(len(dataset))


    # train
    train(
        model = mlp,
        optimizer = optimizer,
        scheduler = scheduler,
        criterion = criterion,
        epochs = epochs,
        device = device,
        train_loader = dataloader_train,
        test_loader = dataloader_test,
        save_weights_name = 'mlp.pth',
        batch_size = batch_size,
        wandb_update = wandb_update
    )

    if wandb_update:
      run.finish()

In [ ]:
!wandb login

In [ ]:
main()

## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib
import matplotlib.colors as mcolors

'''   testing   '''
def test(model, criterion, device, data_loader, dataset, load_weights = False, file_name = None, wandb_update=False):

    if load_weights:
        checkpoint = torch.load(file_name)
        model.load_state_dict(checkpoint['model_state_dict'])
        print('Weights loaded.')


    model.eval()

    testCE = Metric('testCE')

    with torch.no_grad():
        progress_bar = tqdm(data_loader, desc='description')

        overall_preds = []
        overall_targets = []
        softmax = torch.nn.Softmax(dim=1)
        for code, targets in progress_bar:
            code = code.to(device)
            targets = targets.to(device)

            # forward
            outputs = model(code)

            # compute testing loss
            loss = criterion(outputs, targets)

            # see confidence?
            soft = softmax(outputs).cpu()

            # softmax tha outputs and take the max index (argmax), that is the final predicted class
            preds = torch.argmax(softmax(outputs), dim=1).cpu()

            # save results and targets for accuracy metrics
            overall_preds += preds.tolist()
            overall_targets += targets.cpu().tolist()

            testCE.update(loss)

        test_accuracy = accuracy_score(overall_targets, overall_preds)
        f1_w = f1_score(overall_targets, overall_preds, average='weighted')
        f1_m = f1_score(overall_targets, overall_preds, average='micro')
        f1_M = f1_score(overall_targets, overall_preds, average='macro')

        print("predictions:")
        print(overall_preds)
        print("labels")
        print(overall_targets)

        test_log = {'Test CE': testCE.avg.item(),
                      'Test A': test_accuracy}


        geo_weighted = geometric_mean_score(overall_targets, overall_preds, average='weighted')
        geo_micro = geometric_mean_score(overall_targets, overall_preds, average='micro')
        geo_macro = geometric_mean_score(overall_targets, overall_preds, average='macro')

        print("accuracy: ", test_accuracy)
        print("f1 score: ", f1_w)
        print("f1 score macro: ", f1_M)
        print("geo_weighted: ", geo_weighted)
        print("geo_micro: ", geo_micro)
        print("geo_macro: ", geo_macro)



        log('Test CE:{}'.format(test_log['Test CE']))
        log('Test Accuracy:{}'.format(test_log['Test A']))

        print("confusion matrix")
        classes = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']

        cf_matrix = confusion_matrix(overall_targets, overall_preds)

        df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                            columns = [i for i in classes])
        print("data frame")
        plt.figure(figsize = (12,7))
        sn.heatmap(df_cm, annot=True)

        plt.savefig('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/ConfusionMatrix.png')

        plt.show()


        preds = [dataset.int_to_label(i) for i in overall_preds]
        labels = [dataset.int_to_label(i) for i in overall_targets]


        # class accuracy
        acc = []
        n_test = []
        for c in classes:
          correct = 0
          tot = 0

          for i in range(len(labels)):
              if labels[i] == c:
                  if preds[i] == c:
                      correct += 1
                  tot += 1
          try:
              acc_class = correct/tot
          except:
              print("class has no sample")
              acc_class = "Nan"

          acc.append(acc_class)
          n_test.append(tot)


          print("type", acc_class, type(acc_class))


        print("accuracy classes", acc)
        n_tot = [174, 13, 14457, 48, 8, 1019, 67, 4, 198, 895, 31, 23]

        df = pd.DataFrame(list(zip( acc, n_tot, n_test,)), index=[i for i in classes], columns = [ 'Class accuracy', '# samples (tot)', '# samples (test)' ])

        plt.figure(figsize = (12,7))


        norm = mcolors.Normalize(-1,1)

        colors = [[norm(-1.0), "whitesmoke"],
            [norm( 1.0), "whitesmoke"]]

        cmap = mcolors.LinearSegmentedColormap.from_list("", colors)
        hm = sn.heatmap(df, annot=True, cmap=cmap, cbar=False, linecolor="black",fmt='g')


        plt.yticks(rotation='horizontal')
        plt.xticks()

        plt.savefig('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/ClassAccuracy.png')
        plt.show()

In [ ]:
def eval():
    lr = 5e-5
    batch_size = 64
    n_workers = 2
    wandb_update = True
    weight_decay = 1e-3

    latent_space_dim = 512

    seed = 3

    # initialize random seeds
    random_seed(seed)

    mlp = MLP(
        encoder_latent_space = latent_space_dim,
        output_classes = 12
    )

    # extract class weights from csv file
    df_weight = pd.read_csv('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/train_class_weight.csv')
    class_weights = df_weight['weight'].tolist()
    class_weights = torch.FloatTensor(class_weights)

    #criterion = FocalLoss(alpha=class_weights, gamma=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mlp.parameters(), lr = lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                           mode='min',
                                                           factor=0.1,
                                                           patience=8,
                                                           cooldown=0,
                                                           verbose=True)

    device = "cuda:0"
    mlp.to(device)

    # load datasets
    # train dataset
    path_train = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/train.csv'
    dataset_train = CustomDatasetLatent(path_train)
    dataloader_train = DataLoader(dataset_train, shuffle = True, batch_size=batch_size)
    print(len(dataset_train))

    # test dataset
    path_test = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/test.csv'
    dataset_test = CustomDatasetLatent(path_test)
    dataloader_test = DataLoader(dataset_test, shuffle = True, batch_size=batch_size, num_workers=n_workers, pin_memory=True)
    print(len(dataset_test))


    # train
    test(
        model = mlp,
        criterion = criterion,
        device = device,
        data_loader = dataloader_test,
        dataset = dataset_test,
        file_name = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/Checkpoint/Checkpoint.pth',
        load_weights = True,
    )

In [ ]:
eval()